In [ ]:
from flask import Flask, jsonify
from flask_graphql import GraphQLView
import graphene

# Mock data for demonstration purposes
# Replace this with your actual database integration
branches_data = [
    {
        "branch": "Branch 1",
        "bank": {
            "name": "Bank A"
        },
        "ifsc": "IFSC1"
    },
    {
        "branch": "Branch 2",
        "bank": {
            "name": "Bank B"
        },
        "ifsc": "IFSC2"
    },
    # Add more branches as needed
]

class BankType(graphene.ObjectType):
    name = graphene.String()

class BranchType(graphene.ObjectType):
    branch = graphene.String()
    bank = graphene.Field(BankType)
    ifsc = graphene.String()

class Query(graphene.ObjectType):
    branches = graphene.List(BranchType)

    def resolve_branches(self, info):
        # Replace this with your actual database query to fetch branches data
        return branches_data

schema = graphene.Schema(query=Query)

app = Flask(__name__)

# GraphQL API route
app.add_url_rule(
    '/gql',
    view_func=GraphQLView.as_view(
        'graphql',
        schema=schema,
        graphiql=True
    )
)

# REST API endpoints
@app.route('/api/banks', methods=['GET'])
def get_bank_list():
    bank_list = list(set(branch['bank']['name'] for branch in branches_data))
    return jsonify(bank_list)

@app.route('/api/branch/<ifsc>', methods=['GET'])
def get_branch_details(ifsc):
    for branch in branches_data:
        if branch['ifsc'] == ifsc:
            return jsonify(branch)
    return jsonify({'error': 'Branch not found'})

if __name__ == '__main__':
    app.run(debug=True)
